In [11]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
import pandas as pd
import csv
import glob
import math
from math import exp
import random

In [2]:
from audiomentations import Compose, Gain, AddGaussianNoise, TimeMask, AddShortNoises, PolarityInversion, AirAbsorption, RepeatPart, Shift, AdjustDuration

In [3]:
def load_wav_8k_mono(filename):
    try:
        file_contents = tf.io.read_file(filename)
        wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
        sample_rate = tf.cast(sample_rate, dtype=tf.int64)
        if sample_rate != 8000:
            wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=8000)
        wav = tf.squeeze(wav, axis=-1)
    except Exception as e:
        print(f"Invalid file: {filename} - {e}")
    return wav

In [4]:
def preprocess_mel(file_path, label):
    sample_rate = 8000
    mel_spec_length = 32000
    frame_length = 256
    frame_step = 128
    mel_bins = 128
    upper_edge = 4000
    lower_edge = 0
    # load wav and make sure length is proper
    if isinstance(file_path, str):
        try:
            wav, label = load_wav_8k_mono(file_path)
        except Exception as e:
            print(f"Failed to load wav file at {file_path} - {e}")
    else:
        wav = file_path

    if len(wav) > 32000:
        wav = wav[:mel_spec_length]
    if len(wav) < 32000:
        try:
            zero_padding = tf.zeros([mel_spec_length] - tf.shape(wav), dtype=tf.float32)
            wav = tf.concat([zero_padding, wav], 0)
        except Exception as e:
            print(f"Failed to pad wav file at {file_path} - {e}")
    
    norm_wav = (wav - (tf.reduce_min(wav)))/((tf.reduce_max(wav))-(tf.reduce_min(wav))+1e-8)
    #wav = norm_wav
    
    # create the spectrogram
    print("Wav Shape:", wav.shape)
    spectrogram = tf.signal.stft(wav, frame_length, frame_step, window_fn=tf.signal.hann_window)
    print("STFT Shape1:", spectrogram.shape)
    spectrogram = tf.abs(spectrogram)
    print("STFT Shape2:", spectrogram.shape)

    # create the mel filterbank
    spec_bins = frame_length // 2 + 1  # Derived from frame length
    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
        64,
        spec_bins,
        sample_rate,
        lower_edge,
        upper_edge,
    )
    print("Mel Filterbank Shape:", mel_filterbank.shape)
    
    # create the mel spectrogram
    mel_spectrogram = tf.matmul(spectrogram, mel_filterbank)
    print("Mel Spectrogram Shape1:", mel_spectrogram.shape)

    # add a dimension
    mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=2)
    print("Mel Spectrogram Shape2:", mel_spectrogram.shape)

    return mel_spectrogram, label

In [5]:
val_NEG = os.path.join('belize-data', 'Validation data', 'Background')
val_POS = os.path.join('belize-data', 'Validation data', 'Gunshot')

In [24]:
val_pos = glob.glob(val_POS+'/*.wav')
val_pos = tf.data.Dataset.from_tensor_slices((val_pos, [1]*(len(val_pos))))

val_neg = glob.glob(val_NEG+'/*.wav')
val_neg = tf.data.Dataset.from_tensor_slices((val_neg, [0]*(len(val_neg))))

val_joined = val_neg.concatenate(val_pos)

In [25]:
for element in val_joined.take(10):
    print(element)

(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC94E91.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC94EB2.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC94EB5.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC95552.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC95568.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC95755.WAV'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'belize-data\\Validation data\\Background\\5AC95758.WAV'>, <tf.Tenso

In [ ]:
for element in val_joined.take(10):
    element = asnumpy

In [33]:
a = ['a','b','c']
labels = [[0]]
a = tf.data.Dataset.from_tensor_slices((a, [0]*len(a)))
b = ['f','e','d']
b = tf.data.Dataset.from_tensor_slices((b, [1]*len(b)))

c = a.concatenate(b)

c = c.shuffle((len(c)+1))

In [79]:
val_pos = glob.glob(val_POS+'/*.wav')
i=0
for element in val_pos:
    element = val_pos[i]
    label = 0
    val_pos[i] = [element, label]
    i = i+1

val_neg = glob.glob(val_NEG+'/*.wav')
i=0
for element in val_neg:
    element = val_neg[i]
    label = 0
    val_neg[i] = [element, label]
    i = i+1

val_joined = val_pos+val_neg

In [80]:
val_joined

[['belize-data\\Validation data\\Gunshot\\58FBBCC08e94ca6e-256e-4cdc-8bc1-183e50cddc85.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBCC092bcdf2a-27fa-4446-9067-0a957ed49380.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBD00ade05534-429f-4709-8b7a-7206fdc72309.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBD00b53e3fe4-28d8-4828-b280-65277c876a56.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBD406f888e11-66fd-4cbb-9bba-23d3639d5617.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBD40b226dd58-51a2-4cdc-b0be-9114f90f4d31.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBD80fd192559-79ca-4450-89de-82e0db68ee46.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBDC0596cbd27-a153-4903-8a73-d2a8ba97aec6.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBDC0b93e56f7-3277-4757-bb19-6d692827c632.WAV',
  0],
 ['belize-data\\Validation data\\Gunshot\\58FBBE002d1b9316-1ec1-447b-aa79-4e8590157944.WAV',
  0],
 ['belize-

In [ ]:
def create_results():
    for element in val_joined:
        filepath = element[0]
        label = element[1]
        wav1 = load_wav_8k_mono(filepath)
        wav2 = load_and_augment_far(filepath)
        wav3 = load_and_augment_close(filepath)
        mel_spec1 = preprocess_mel(wav1)
        mel_spec1 = mel_spec1[np.newaxis(), :]
        mel_spec2 = preprocess_mel(wav2)
        mel_spec2 = mel_spec1[np.newaxis(), :]
        mel_spec3 = preprocess_mel(wav3)
        mel_spec3 = mel_spec3[np.newaxis(), :]
        interpreter.set_tensor(input_details[0]['index'], mel_spec1)
        interpreter.invoke()
        yhat1 = interpreter.get_tensor(output_details[0]['index'])[0]
        interpreter.set_tensor(input_details[0]['index'], mel_spec2)
        interpreter.invoke()
        yhat2 = interpreter.get_tensor(output_details[0]['index'])[0]
        interpreter.set_tensor(input_details[0]['index'], mel_spec3)
        interpreter.invoke()
        yhat3 = interpreter.get_tensor(output_details[0]['index'])[0]
        combined_pred = combine_preds(yhat1, yhat2, yhat3, 0)
        sigmoided_pred = sigmoid_pred(combined_pred)
        results.append([filepath, label, yhat1, yhat2, yhat3, combined_pred1, combined_pred2, sigmoided_pred1, sigmoided_pred2])
    return results

In [26]:
def combine_preds(p1, p2, p3, square):
    if square == 1:
        combined_pred = (1-(((1-(p1**2))*(1-(p2**2))+(1-(p2**2))*(1-(p3**2))+(1-(p1**2))*(1-(p3**2)))/(p1+p2+p3+1e-06)))
    else:
        combined_pred = (1-(((1-p1)*(1-p2)+(1-p2)*(1-p3)+(1-p1)*(1-p3))/(p1+p2+p3+1e-06)))
    return combined_pred

In [39]:
def sigmoid_pred(pf):
    x = (-10)*(pf-0.5)
    sigmoided_pred = 1/(1+exp(x))
    return sigmoided_pred

In [43]:
combine_preds(0.8,0.6,0.35,1)

0.36691464747734437

In [44]:
sigmoid_pred(0.36691464747734437)

0.20901821718000488

In [6]:
a = [1,2,3]
for i in range(len(a)):
    a = a+a

In [7]:
a

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [74]:
array = ['a','b','c']

i=0
for element in array:
    element = array[i]
    label = 0
    array[i] = [element, label]
    i = i+1

In [75]:
array

[['a', 0], ['b', 0], ['c', 0]]

In [67]:
a = ['a','b','c']
a = tf.data.Dataset.from_tensor_slices((a, tf.zeros(len(a))))
b = ['f','e','d']
b = tf.data.Dataset.from_tensor_slices((b, tf.ones(len(b))))

c = a.concatenate(b)

c = c.shuffle((len(c)+1))

In [68]:
c = c.as_numpy_iterator().next()

In [69]:
c

(b'c', 0.0)

In [37]:
c = c.numpy()

AttributeError: 'ShuffleDataset' object has no attribute 'numpy'

In [34]:
for element in c:
    element = np.asarray(element)
    label =
    print(element)

[<tf.Tensor: shape=(), dtype=string, numpy=b'f'>
 <tf.Tensor: shape=(), dtype=int32, numpy=1>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'b'>
 <tf.Tensor: shape=(), dtype=int32, numpy=0>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'a'>
 <tf.Tensor: shape=(), dtype=int32, numpy=0>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'e'>
 <tf.Tensor: shape=(), dtype=int32, numpy=1>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'c'>
 <tf.Tensor: shape=(), dtype=int32, numpy=0>]
[<tf.Tensor: shape=(), dtype=string, numpy=b'd'>
 <tf.Tensor: shape=(), dtype=int32, numpy=1>]


In [39]:
for element in c:
    element = np.asarray(element)
    element = element.numpy()
    print(element)

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [42]:
# Create a tensor
tensor = tf.constant([[1, 2], [3, 4]])
print(tensor)
# Convert the tensor to a NumPy array
numpy_array = tensor.numpy()

# Print the NumPy array
print(numpy_array)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
[[1 2]
 [3 4]]


In [44]:
tensor[1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4])>